In [11]:
import itertools
import random
import simpy

## Utils

### Class

In [12]:
class My_container(simpy.Container):
    def __init__(self, env, name: str, capacity: int | float = ..., init: int | float = 0):
        super().__init__(env, capacity, init)
        self.name = name


class My_resource(simpy.Resource):
    def __init__(self, env, name: str, container: My_container=None, capacity: int = 1):
        super().__init__(env, capacity)
        self.name = name
        self.container = container

class Resources_set:
    def __init__(self, env, count_resources, name, container: My_container=None):
        self.env = env
        self.resources = [My_resource(env, f'{name}_{i}', container, capacity=1) for i in range(count_resources)]

    def request_resource(self):
        # Genera un evento para cada recurso
        for resource in self.resources:
            yield resource.request()  # Genera un evento de solicitud

        # Genera un evento que nunca se completa (para el "break")
        yield self.env.event()
        

    def release_resource(self, resource):
        resource.release(resource)


### Functions

In [13]:
def rule_of_three(numerator, denominator, factor):
    return (numerator/denominator)*factor

## Definiendo parámetros

In [35]:
""" SIMULATION """
RANDOM_SEED = 42
T_INTER = [30, 300]            # intervalo entre la llegada de los turistas
SIM_TIME = 500                # tiempo total de la simulación

""" ROOM """
ROOM_CLEANING_SIZE = 50       # máximo nive de limpieza de una habitación 
THRESHOLD_CLEAN = 50           # mínimo de limpieza/confort (% del total)

""" TOURIST """
TOURIST_ENERGY_SIZE = 50       # nivel máximo de descanso
TOURIST_ENERGY_LEVEL = [5, 25] # nivel de energía de los turistas (menor_energía => más_sueño)
SLEEP_SPEED = 2                # tasa de recuperar energía (u / second)

""" WORKER """
HOUSEMAID_TIME = 50           # tiempo que tarda la mucama en limpiar la habitación (segundos)


""" MONEY """
PRICES = {}
PRICES['room'] = 50

SALARIES = {}
SALARIES['housemaid'] = 5

SALARIES_AMOUNT = {}           # salario cobrado por cada trabajador
SALARIES_AMOUNT['housemaid'] = 0

AMOUNT = {}                    # pago acumulado por Los turistas en cada servicio
AMOUNT['room'] = 0
#AMOUNT['room'] = (0, 0, 0)    # (pago_del_turista, del_1ro_lo_pagado_al_trabajador, cant_turist)

### Red social

In [36]:
SATISFACTION = {} # ['tourist_name'] = {['necesity'] = level_when_leaves_the_hotel}

# Agents
## Tourist

In [32]:
def tourist(name, env, rooms): # most be include 'age'
   
    tourist_energy_level = random.randint(*TOURIST_ENERGY_LEVEL) 

    print(f'{env.now:6.1f} s: {name} arrived at room')

    for room in rooms.resources:
        if room.count == 0:
            # Solicita un recurso 
            with room.request() as request_room:
                yield request_room
                _room = rooms.resources[rooms.resources.index(request_room.resource)]
                print(f"El turista {name} accedió a la habitación {_room.name}")

                # Get the required amount of energy
                energy_required = TOURIST_ENERGY_SIZE - tourist_energy_level

                if _room.container.level < energy_required:
                    print(f'El turista {name} no pudo descansar. La habitación {_room.name} estaba DEMASIADO sucia. Se ha ido SIN PAGAR')
                    print(f'Level of clean: {_room.container.level}')

                    if name not in SATISFACTION:
                        SATISFACTION[name] = {}        
                    SATISFACTION[name]['energy'] = tourist_energy_level
                    print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
                    print(f'Actual level of clean of {_room.name}: {_room.container.level}')
                    break

                else:
                    AMOUNT['room'] += PRICES['room']
                    yield _room.container.get(energy_required)

                    # The "actual" recovering of energy process takes some time
                    yield env.timeout(energy_required / SLEEP_SPEED)

                    if name not in SATISFACTION:
                        SATISFACTION[name] = {}        
                    SATISFACTION[name]['energy'] = tourist_energy_level + energy_required
                    print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
                    print(f'Actual level of clean of {_room.name}: {_room.container.level}')

                    break


## Workers

In [33]:
def housemaid(env, room):
    """Arrives at the room and finish of clean it after a certain delay."""
    #add si no tiene el salario completo no rellena al máximo el nivel de limpieza
    yield env.timeout(2)

    with room.request() as rq:
        yield rq
        print(f'{env.now:6.1f} s: Housemaid is cleaning the {room.container.name} of the {room.name}...')
        bed = room.container
        amount = bed.capacity - bed.level
        print(f'{env.now:6.1f} s: Level before clean the {room.name}: {bed.level}')
        bed.put(amount)
        yield env.timeout(HOUSEMAID_TIME)

    print(
        f'{env.now:6.1f} s: Housemaid finished and the room is clean'
    )
    print(f'Level after clean {room.name}: {bed.level}')

## Manager

In [25]:
def manager(env, rooms):
    """Periodically check the clean level of the room and call the housemaid
       if the level falls below a threshold."""
    while True:
        # DO SOMETHING FOR VERIFY THE EVOLUTION OF THE AMOUNT%%%%%%%%%%%%%%%%%%%%$$$$$$$$$$$$$$$$###################
        # 


        # actual_amount = AMOUNT
        # yield env.timeout(30)
        # diff = DeepDiff(actual_amount, AMOUNT)
        # print(f'Actualization of evolution of amount: {diff}')

        for room in rooms:

            if room.container.level / room.container.capacity * 100 < THRESHOLD_CLEAN:
                # We need to call the housemaid now!
                
                # Wait for the housemaid to clean the room
                yield env.process(housemaid(env, room))

                housemaid_salary = SALARIES['housemaid']

                if AMOUNT['room'] >= housemaid_salary:
                    AMOUNT['room'] -= housemaid_salary
                    SALARIES_AMOUNT['housemaid'] += housemaid_salary
                    print(f'{env.now:6.1f} s: The housemaid charaged ${housemaid_salary}. The amount salary of housemaid is {SALARIES_AMOUNT['housemaid']}')

                # else: (algo con el rendimiento de la housemaid al limpiar la habitación)
        
        yield env.timeout(5)  # Check every 10 seconds

        # añadir análisis con las ganancias

In [19]:
def tourist_generator(env, rooms):
    """Generate new tourists that arrive at the hotel."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(tourist(f'Tourist {i}', env, rooms))

# Simulation

In [37]:
# Setup and start the simulation
print('Hotel is open')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()  # hotel

rooms = Resources_set(env, 10, 'room', My_container(env, 'bed', ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE))


env.process(manager(env, rooms.resources))
env.process(tourist_generator(env, rooms))

# Execute!
env.run(until=SIM_TIME)


Hotel is open
  87.0 s: Tourist 0 arrived at room
El turista Tourist 0 accedió a la habitación room_0
Actual level of satisfaction: {'Tourist 0': {'energy': 50}}
 Actual amount: {'room': 50}
Actual level of clean of room_0: 13
 105.5 s: Housemaid is cleaning the bed of the room_0...
 105.5 s: Level before clean the room_0: 13
 129.0 s: Tourist 1 arrived at room
El turista Tourist 1 accedió a la habitación room_1
Actual level of satisfaction: {'Tourist 0': {'energy': 50}, 'Tourist 1': {'energy': 50}}
 Actual amount: {'room': 100}
Actual level of clean of room_1: 12
 155.5 s: Housemaid finished and the room is clean
Level after clean room_0: 12
 155.5 s: The housemaid charaged $5. The amount salary of housemaid is 5
 157.5 s: Housemaid is cleaning the bed of the room_1...
 157.5 s: Level before clean the room_1: 12
 207.5 s: Housemaid finished and the room is clean
Level after clean room_1: 50
 207.5 s: The housemaid charaged $5. The amount salary of housemaid is 10
 284.0 s: Tourist 2 a